In [ ]:
!pip install -q -U google-generativeai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import pandas as pd
from PIL import Image
from tqdm import tqdm
import time  # Import time module for delay
import google.generativeai as genai

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import google.generativeai as genai
model = genai.GenerativeModel('gemini-1.5-flash-latest')

In [ ]:
# Function to generate description for an image
def generate_description(image_path):
    try:
        img = Image.open(image_path)
        prompt = '''Based on the following details generate description for provided image
        Visuals: Detail key elements like shapes, lines, and textures, specifying the type (e.g., geometric, abstract).
        Colors: Describe primary and secondary colors, and any thematic palette (e.g., pastels).
        Style: Note artistic influences or styles (e.g., Art Deco, modern).
        Mood: Describe the emotional tone (e.g., soothing, energetic).
        Category: Classify the pattern.
        Uniqueness: Highlight distinctive features.
        provide the output in way that we can use that description to generate an image with image generation models It should be less than 100 tokens.
        ...'''
        response = model.generate_content([prompt, img], stream=False)
        response.resolve()
        description = response.text
        img.close()
        return description
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

In [ ]:
# Set the seed for reproducibility
seed = 42
random.seed(seed)

# Define the directory containing images
image_directory = "/content/drive/MyDrive/abstract_2024-05-09_0519"
all_images = [os.path.join(image_directory, img) for img in os.listdir(image_directory) if img.endswith(('.png', '.jpg', '.jpeg'))]

In [ ]:
selected_images = random.sample(all_images, 100)

In [ ]:
# Create a DataFrame to store image names and descriptions
data = []

# Iterate over the selected images and generate descriptions
for image_path in tqdm(selected_images, desc="Processing Images"):
    description = generate_description(image_path)
    if description:
        data.append({"Image Name": os.path.basename(image_path), "Description": description})
    time.sleep(2)  # Add a 2-second delay between requests

Processing Images:  93%|█████████▎| 93/100 [14:08<00:52,  7.50s/it]ERROR:tornado.access:500 POST /v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 7014.42ms


Error processing image /content/drive/MyDrive/abstract_2024-05-09_0519/5031255 -3.jpg: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting


Processing Images: 100%|██████████| 100/100 [15:00<00:00,  9.00s/it]


In [ ]:
# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
output_csv_path = "/content/drive/MyDrive/gemini-flash_image_descriptions.csv"
df.to_csv(output_csv_path, index=False)
print(f"Descriptions saved to {output_csv_path}")

Descriptions saved to /content/drive/MyDrive/gemini-flash_image_descriptions.csv


In [ ]:
# Optional: Print a sample of the DataFrame
df.head()

,Image Name,Description
0,5030930-5.jpg,A seamless pattern with a swirling abstract de...
1,50310112-4.jpg,A seamless pattern with abstract shapes and li...
2,4031065.jpg,An abstract pattern with geometric shapes and ...
3,10301077-2.jpg,"Abstract pattern with organic, textured shapes..."
4,5031120-4.jpg,"Abstract pattern with green, gray, and black c..."
